In [2]:
%matplotlib qt
from matplotlib import pyplot as plt
from numpy.random import rand
import skimage as sm
import skimage.morphology as morph
from skimage.io import imread, imsave
from skimage.util import img_as_float, random_noise
from skimage.transform import rotate, resize
from skimage.filters import gaussian as ski_gaussian
from pylab import ginput
from scipy.signal import convolve, convolve2d, correlate2d, fftconvolve
from scipy.signal import  gaussian as scipy_gaussian
from scipy.fft import fft2, fftshift, ifft2, ifftshift
from scipy.interpolate import griddata
from scipy.ndimage import map_coordinates

import numpy as np
import os
import timeit


os.chdir("../Mats")

In [ ]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# 1

1.1

1.1.1

In [3]:
A = imread("cells_binary_inv.png")

disk = morph.disk(1)
openA = morph.binary_opening(A)
closeA = morph.binary_closing(A)

plt.subplot(1,3,1)
plt.imshow(A,cmap="gray")
plt.subplot(1,3,2)
plt.imshow(openA,cmap="gray")
plt.subplot(1,3,3)
plt.imshow(closeA,cmap="gray")


1.1.2

In [ ]:
A = imread("money_bin.jpg")

A = imread("cells_binary_inv.png")


mask = A >230
A[mask] = 255
A[~mask] =0


disk = morph.disk(1)

Atest = morph.binary_closing(A)
Atest = morph.binary_opening(Atest)

Atest = Atest.astype(np.uint8)



plt.subplot(1,3,1)
plt.imshow(A,cmap="gray")
plt.subplot(1,3,2)
plt.imshow(Atest,cmap="gray")

mask = Atest == True
Atest[mask] = 255
Atest[~mask] =0



plt.subplot(1,3,3)
plt.imshow(Atest,cmap="gray")


In [ ]:
connectedA = sm.measure.label(Atest, background=255,connectivity=2)

plt.subplot(1,2,1)
plt.imshow(A,cmap="gray")
plt.subplot(1,2,2)
plt.imshow(connectedA,cmap="gray")

In [ ]:
connectedA.max()

1.1.3

1.2

In [ ]:
A = imread("money_bin.jpg")

mask = A >230
A[mask] = 0
A[~mask] =1


disk = morph.disk(1)

Atest = morph.binary_closing(A)
Atest = morph.binary_opening(Atest)

connectedA = sm.measure.label(Atest, background=255,connectivity=2)
plt.imshow(connectedA,cmap="jet")


In [5]:
plt.imshow(connectedA,cmap="jet")

In [21]:
def check_positions(m1, m2, N):
    mask = np.array(m1) == N
    result = np.any(np.array(m2)[mask] != 0)
    if result:
        print(result)
        print(f"Label {N} is gone")

    return result

A = imread("money_bin.jpg")
mask = A > 230
A[mask] = 0
A[~mask] = 1

disk = morph.disk(1)
Atest = morph.binary_closing(A)
Atest = morph.binary_opening(Atest)


connectedA = sm.measure.label(Atest, background=255, connectivity=2)

Loops = 30
fig, axs = plt.subplots(nrows=(Loops-1)//3+1, ncols=3, figsize=(50, 50), dpi=500)
uq = np.unique(connectedA)
uq = uq[1:]
for i in range(Loops):
    axs[i//3, i%3].imshow(Atest, cmap="gray")
    axs[i//3, i%3].axis("off")
    Atest = morph.binary_erosion(Atest, disk)
    for j in uq:
        if check_positions(connectedA, Atest, j):
            print(i)
            connectedA[connectedA == j] = 0

fig.subplots_adjust(wspace=0)

plt.show()

True
Label 1 is gone
0
True
Label 2 is gone
0
True
Label 3 is gone
0
True
Label 4 is gone
0
True
Label 5 is gone
0
True
Label 6 is gone
0
True
Label 7 is gone
0
True
Label 8 is gone
0
True
Label 9 is gone
0
True
Label 10 is gone
0
True
Label 11 is gone
0


# 2

2.1

In [ ]:
def LSI(img,kernel,noise):
    imgFFT = fft2(img)
    kernelFFT = fft2(kernel)



kernel = []
noise = []

filtA = LSI(A,kernel,noise)
plt.subplot(1,2,1)
plt.imshow(A,cmap="gray")
plt.subplot(1,2,2)
plt.imshow(filtA,cmap="gray")

2.2

2.3

# 3

3.4

In [4]:
def centred_square(x, y, N):
    if x%2==0 or y%2==0:
        raise ValueError("The x and y dimensions of the image must be odd")
    output = np.zeros((x, y))
    x_center, y_center = x//2, y//2
    half_N = N//2
    if N%2==1:
        output[x_center-half_N:x_center+half_N+1, y_center-half_N:y_center+half_N+1] = 1
    else:
        output[x_center-half_N:x_center+half_N, y_center-half_N:y_center+half_N] = 1
    return output

plt.imshow(centred_square(31, 31, 9), cmap="gray")
plt.show()

3.5

In [5]:
def filter_translation(image, tx, ty, mode="full", boundary="fill"):
    filter_x, filter_y = np.zeros(2*np.abs(tx)+1), np.zeros(2*np.abs(ty)+1)
    filter_x[np.abs(tx)+tx] = 1
    filter_y[np.abs(ty)+ty] = 1
    filter = np.outer(filter_y, filter_x)
    output = convolve2d(image, filter, mode, boundary)
    return output
A = centred_square(9, 9, 3)
plt.subplot(1, 2, 1)
plt.imshow(A, cmap="gray")
B = filter_translation(A, 3, 3, mode="same")
plt.subplot(1, 2, 2)
plt.imshow(B, cmap="gray")
plt.show()

3.6

In [23]:
def homogeneous_translation(image, tx, ty):
        output = np.zeros(image.shape)
        matrix = np.identity(3)
        matrix[0, 2] = tx
        matrix[1, 2] = ty
        matrix = np.linalg.inv(matrix)
        x = np.linspace(1, image.shape[1], image.shape[1])
        y = np.linspace(1, image.shape[0], image.shape[0])
        for i in range(len(y)):
            for j in range(len(x)):
                loc = matrix@np.array([x[i], y[j], 1])

                x_nearest = int(round(loc[0]-1))
                y_nearest = int(round(loc[1]-1))
                if x_nearest >= image.shape[0] or y_nearest >= image.shape[1]:
                    output[j, i] = 0
                else:    
                    output[j, i] = image[x_nearest,y_nearest]
        return output
    
    
A = centred_square(9, 9, 3)
plt.subplot(1, 2, 1)
plt.imshow(A, cmap="gray")
B = homogeneous_translation(A, 3, 3)
print(B)
plt.subplot(1, 2, 2)
plt.imshow(B, cmap="gray")
plt.show()

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1.]]


3.7

In [ ]:
def fourier_translation(image, tx, ty):
    fft_image = fft2(image)
    u = np.fft.fftfreq(fft_image.shape[1])
    v = np.fft.fftfreq(fft_image.shape[0])
    u, v = np.meshgrid(u, v)
    rotate = np.exp(-2j*np.pi*(u*tx+v*ty))
    return np.real(ifft2(fft_image*rotate))

A = centred_square(15, 15, 5)
plt.subplot(1, 3, 1)
plt.imshow(A, cmap="gray")
B = filter_translation(A, 2, 3, mode="same")
plt.subplot(1, 3, 2)
plt.imshow(B, cmap="gray")
C = fourier_translation(A, 2, 3)
plt.subplot(1, 3, 3)
plt.imshow(C, cmap="gray")
plt.show()

3.8